In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import LinuxImage

img = LinuxImage.from_str("5.4.0-26-generic-amd64")

symtab = img.symtab

[   symtab.py:31 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-amd64.pkl


In [3]:
local_func_df = symtab.funcs_local

len(local_func_df)

31504

In [4]:
renamed_df = symtab.funcs_renamed

renamed_df

,name,section,bind,type,visibility,value,size
112,sanitize_boot_params.constprop.0,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071578853384,160
171,memblock_alloc.constprop.0,.init.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071604652691,24
257,name_to_dev_t.cold,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071578857585,51
258,create_dev.constprop.0,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071578857636,83
360,calibrate_delay.cold,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071578859109,534
...,...,...,...,...,...,...,...
90777,insn_get_opcode.part.0,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071590126640,408
90778,insn_get_modrm.part.0,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071590127056,226
90779,insn_get_sib.part.0,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071590127296,96
90780,insn_get_displacement.part.0,.text,STB_LOCAL,STT_FUNC,STV_DEFAULT,18446744071590127392,208


In [5]:
suffix_df = renamed_df.name.map(
    lambda x: x.split(".", 1)[-1].replace(".0", "")
).value_counts()

suffix_df = suffix_df.reset_index()
suffix_df.columns = ["suffix", "count"]
suffix_df["renamed %"] = suffix_df["count"] / len(renamed_df) * 100
suffix_df["local %"] = suffix_df["count"] / len(local_func_df) * 100


print(suffix_df["local %"].sum())

suffix_df

17.270822752666326


,suffix,count,renamed %,local %
0,cold,3228,59.327330,10.246318
1,isra,783,14.390737,2.485399
2,part,774,14.225326,2.456831
3,constprop,409,7.517001,1.298248
4,isra.cold,73,1.341665,0.231717
5,isra.part,59,1.084359,0.187278
6,part.cold,58,1.065981,0.184104
7,constprop.cold,29,0.532990,0.092052
8,isra.constprop,19,0.349201,0.060310
9,isra.part.cold,4,0.073516,0.012697


In [6]:
import pandas as pd

combined = suffix_df["suffix"].str.contains(r"\.")

plot_df = pd.concat(
    [
        suffix_df[~combined],
        pd.DataFrame(
            {
                "suffix": "≥2",
                **suffix_df[combined].sum(numeric_only=True),
            },
            index=[0]
        )
    ],
    ignore_index=True
)

plot_df

,suffix,count,renamed %,local %
0,cold,3228.0,59.327330,10.246318
1,isra,783.0,14.390737,2.485399
2,part,774.0,14.225326,2.456831
3,constprop,409.0,7.517001,1.298248
4,≥2,247.0,4.539607,0.784027


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(2.5, 2.5), dpi=300)


total = plot_df["count"].sum()

ax.pie(
    plot_df["count"],
    labels=plot_df["suffix"],
    autopct=lambda pct: f"{pct:.1f}%\n({int(pct * total / 100)})",
    textprops={'fontsize': 6}
)

plt.show()